In [36]:
%load_ext autoreload
%autoreload 2
import numpy as np
from src import GetUserClimbData as gud
from src import AssesItemBased as assess
from sklearn.metrics.pairwise import cosine_similarity

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
uid_path = './user_ids/uids.csv'
user_data = gud.get_users_climbs(uid_path)
user_data_idx = user_data.set_index('user_id')
# route_data = star_data.T  
# route_data = route_data.drop(labels='user_id', axis=0)
# cs_sim = cosine_similarity(route_data)

In [19]:
route_data = user_data.T  
route_data = route_data.drop(labels='user_id', axis=0)
cs_sim = cosine_similarity(route_data)


In [24]:
test_split = user_data_idx[:6]
train_split = user_data_idx[6:]
test_split.shape, train_split.shape

((6, 3590), (24, 3590))

In [42]:
held_out_routes, test_trim = assess.trim_routes(test_split,0.2)
train_held_out = train_split.append(test_trim)
train_held_out.shape

(30, 3590)

In [46]:
recs_fold_1 = assess.get_predictions(
    train_held_out,
    test_split,
    train_held_out.T,
    cs_sim)

In [56]:
assess.calc_map(recs_fold_1,held_out_routes,test_split)

0.26909167221667224
0.04854457726030626
0.10032542924146753
0.07890671679064626
0.11368289184110006
0.08304354801501328


0.11559913922753427

In [67]:
judge, base = assess.format_for_galago(recs_fold_1,held_out_routes)
assess.export_data(judge,base,1)

In [66]:
base[:30]

['105750112 0 106503119 1 27.9276',
 '106228490 0 106503119 1 27.9276',
 '106590813 0 106503119 1 27.9276',
 '105793864 0 106503119 1 27.9276',
 '109639920 0 106503119 1 27.9276',
 '106926010 0 106503119 1 27.9276',
 '105762414 0 106503119 1 27.9276',
 '106538566 0 106503119 1 27.9276',
 '105762411 0 106503119 1 27.9276',
 '106858564 0 106503119 1 27.9276',
 '106900839 0 106503119 1 27.9276',
 '106891377 0 106503119 1 27.9276',
 '105748050 0 106503119 1 27.9276',
 '105985613 0 106503119 1 27.9276',
 '105850436 0 106503119 1 27.9276',
 '105863136 0 106503119 1 27.9276',
 '105763875 0 106503119 1 27.9276',
 '105748556 0 106503119 1 27.9276',
 '105748529 0 106503119 1 27.9276',
 '105753172 0 106503119 1 27.9276',
 '111850835 0 106503119 1 27.9276',
 '105760938 0 106503119 1 27.9276',
 '105756925 0 106503119 1 27.9276',
 '105756799 0 106503119 2 27.3126',
 '105750154 0 106503119 3 27.1525',
 '105749188 0 106503119 4 26.6781',
 '105750013 0 106503119 5 26.5835',
 '105748951 0 106503119 6 26